# Plot literature review of conductivities and permittivities

The original Gabriel model (https://doi.org/10.1088/0031-9155/41/11/003) is compared against an algorithmic correction by Zimmermann and van Rienen (https://doi.org/10.1016/j.bioelechem.2021.107773).

**TODO: The data by Wagner et al. has not yet been added (Supp. Fig. 2 of https://doi.org/10.1016/j.neuroimage.2013.06.079).**

The parameter values of the original Gabriel model are available online: http://niremf.ifac.cnr.it/docs/DIELECTRIC/AppendixC.html

This website is also archived here: https://web.archive.org/web/20240718162705/http://niremf.ifac.cnr.it/docs/DIELECTRIC/AppendixC.html

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.constants import epsilon_0 as e0
from brainsignals.plotting_convention import simplify_axes

## 4 Cole-Cole model

This is an implementation of the 4 Cole-Cole model of the complex permittivity suggested by Gabriel et al.
The source code is mostly taken from ImpedanceFitter (https://github.com/j-zimmermann/impedancefitter).

In a helper function, the complex permittivity can be converted into relative permittivity and conductivity.

In [ ]:
def cole_cole_4_model(
    omega,
    epsinf,
    deps1,
    deps2,
    deps3,
    deps4,
    tau1,
    tau2,
    tau3,
    tau4,
    a1,
    a2,
    a3,
    a4,
    sigma,
):
    r"""Standard 4-Cole-Cole impedance model.


    Parameters
    ----------
    omega: :class:`numpy.ndarray`, double
        list of frequencies
    epsinf: double
        value for :math:`\varepsilon_\infty`
    deps1: double
        value for :math:`\Delta\varepsilon_1`
    deps2: double
        value for :math:`\Delta\varepsilon_2`
    deps3: double
        value for :math:`\Delta\varepsilon_3`
    deps4: double
        value for :math:`\Delta\varepsilon_4`
    tau1: double
        value for :math:`\tau_1`, in ps
    tau2: double
        value for :math:`\tau_2`, in ns
    tau3: double
        value for :math:`\tau_3`, in us
    tau4: double
        value for :math:`\tau_4`, in ms
    a1: double
        value for :math:`1 - \alpha_1 = a`
    a2: double
        value for :math:`1 - \alpha_2 = a`
    a3: double
        value for :math:`1 - \alpha_3 = a`
    a4: double
        value for :math:`1 - \alpha_4 = a`
    sigma: double
        conductivity value

    Returns
    -------
    :class:`numpy.ndarray`, complex
        Complex permittivity array

    Notes
    -----
    The original model has been described in [Gabriel1996]_.

    References
    ----------
    .. [Gabriel1996] Gabriel, S., Lau, R. W., & Gabriel, C. (1996).
                    The dielectric properties of biological tissues:
                    III. Parametric models for the dielectric spectrum of tissues.
                    Physics in Medicine and Biology, 41(11), 2271–2293.
                    https://doi.org/10.1088/0031-9155/41/11/003
    """
    tau1 *= 1e-12
    tau2 *= 1e-9
    tau3 *= 1e-6
    tau4 *= 1e-3
    epsc = epsinf - 1j * sigma / (omega * e0)

    epsc += deps1 / (1.0 + np.power((1j * omega * tau1), a1))
    epsc += deps2 / (1.0 + np.power((1j * omega * tau2), a2))
    epsc += deps3 / (1.0 + np.power((1j * omega * tau3), a3))
    epsc += deps4 / (1.0 + np.power((1j * omega * tau4), a4))
    return epsc

def return_diel_properties(omega, epsc):
    r"""Return relative permittivity and conductivity from complex permittivity.

    Notes
    -----

    The relative permittivity is the real part of :math:`\varepsilon_\mathrm{r}^\ast`
    and the conductivity is the negative imaginary part times the frequency and the
    vacuum permittivity (see, e.g., [Grant1958]_.

    Parameters
    ----------
    omega: :class:`numpy.ndarray`, double
        frequency array
    epsc: :class:`numpy.ndarray`, complex
        complex permittivity array

    Returns
    -------
    eps_r: :class:`numpy.ndarray`, double
        relative permittivity
    conductivity: :class:`numpy.ndarray`, double
        conductivity in S/m

    References
    ----------
    .. [Grant1958] Grant, F. A. (1958).
                   Use of complex conductivity in the
                   representation of dielectric phenomena.
                   Journal of Applied Physics, 29(1), 76–80.
                   https://doi.org/10.1063/1.1722949
    """
    eps_r = epsc.real
    conductivity = -epsc.imag * e0 * omega
    return eps_r, conductivity

# Plots of the dielectric properties of brain tissue

The dielectric properties are plotted between 1 Hz and 10 kHz.
Grey and white matter data are plotted.

The Zimmermann / van Rienen model removes the fourth dispersion and adjusts the Gabriel model.
Otherwise, all parameter values are the same as in the Gabriel model.

In [ ]:
frequencies = np.logspace(1, 4)
omega = 2.0 * np.pi * frequencies

In [ ]:
# Gray matter data

review_dict = {}
review_dict['Gabriel et al. (1996)'] = {"epsinf": 4.0, 
                                  "deps1": 45, 
                                  "tau1": 7.958,
                                  "a1": 0.9,
                                  "deps2": 400, 
                                  "tau2": 15.915,
                                  "a2": 0.85,
                                  "sigma": 0.02,
                                  "deps3": 200000.0, 
                                  "tau3": 106.103, 
                                  "a3": 0.78,
                                  "deps4": 45000000.0,
                                  "tau4": 5.305,
                                  "a4": 1.0
                                  }

review_dict["Zimmermann, van Rienen (2021)"] = review_dict["Gabriel et al. (1996)"].copy()
review_dict["Zimmermann, van Rienen (2021)"]["deps4"] = 0
review_dict["Zimmermann, van Rienen (2021)"]["sigma"] = 0.10697690700220125

fig = plt.figure(figsize=[6, 6])
fig.subplots_adjust(bottom=0.17, left=0.1, right=0.7)
ax1 = fig.add_subplot(311,
                      ylabel='S/m',
                     ylim=[0.0, 0.7]
                     )
ax1.grid(True)
ax2= fig.add_subplot(312, 
                      sharex=ax1,
                     ylim=[0, 4e7]
                     )
ax2.grid(True)
ax3 = fig.add_subplot(313, 
                      sharex=ax1,
                    xlabel='$f$ (Hz)',
                      ylim=[0, 1]
                     )
ax3.grid(True)

keys = review_dict.keys()

for key in keys:
    parameters = review_dict[key]
    factor = 1
    if "Zimmermann" in key:
        factor = 100
        ax2.text(1e3, 1.5e7, "100x", color="tab:orange")

    epsc = cole_cole_4_model(omega, **parameters)
    eps_r, conductivity = return_diel_properties(omega, epsc)
    ax1.semilogx(frequencies, conductivity, label=key, lw=2)
    ax2.semilogx(frequencies, factor * eps_r, label=key, lw=2)
    ax3.semilogx(frequencies, omega* e0* eps_r / conductivity, label=key, lw=2)

ax1.set_title(r"$\sigma_\mathrm{rt}$")
ax2.set_title(r"$\varepsilon_\mathrm{rt}$")
ax2.legend()
ax3.set_title(r"$2\pi\varepsilon_\mathrm{rt}/\sigma_\mathrm{rt}$")

#fig.legend(lines, line_names, frameon=False, ncol=1, loc='right')
#simplify_axes(ax1)

plt.savefig("gray_matter_review.pdf")

In [ ]:
review_dict = {}
# White matter data
review_dict['Gabriel et al. (1996)'] = {"epsinf": 4.0, 
                                 "deps1": 32,
                                 "tau1": 7.958,
                                 "a1": 0.9,
                                 "deps2": 100,
                                 "tau2": 7.958,
                                 "a2": 0.9,
                                 "sigma": 0.02,
                                 "deps3": 40000.0,
                                 "tau3": 53.052,
                                 "a3": 0.7,
                                 "deps4": 35000000.0,
                                 "tau4": 7.958,
                                 "a4": 0.98}

review_dict["Zimmermann, van Rienen (2021)"] = review_dict["Gabriel et al. (1996)"].copy()
review_dict["Zimmermann, van Rienen (2021)"]["deps4"] = 0
review_dict["Zimmermann, van Rienen (2021)"]["sigma"] = 0.061080374691163

fig = plt.figure(figsize=[6, 6])
fig.subplots_adjust(bottom=0.17, left=0.1, right=0.7)
ax1 = fig.add_subplot(311,
                      ylabel='S/m',
                     ylim=[0.0, 0.3]
                     )
ax1.grid(True)
ax2= fig.add_subplot(312, 
                      sharex=ax1,
                     ylim=[0, 3e7]
                     )
ax2.grid(True)
ax3 = fig.add_subplot(313, 
                      sharex=ax1,
                    xlabel='$f$ (Hz)',
                      ylim=[0, 1]
                     )
ax3.grid(True)

keys = review_dict.keys()

for key in keys:
    parameters = review_dict[key]
    factor = 1
    if "Zimmermann" in key:
        factor = 100
        ax2.text(1e3, 0.5e7, "100x", color="tab:orange")
    epsc = cole_cole_4_model(omega, **parameters)
    eps_r, conductivity = return_diel_properties(omega, epsc)
    ax1.semilogx(frequencies, conductivity, label=key, lw=2)
    ax2.semilogx(frequencies, factor * eps_r, label=key, lw=2)
    ax3.semilogx(frequencies, omega* e0* eps_r / conductivity, label=key, lw=2)


ax1.set_title(r"$\sigma_\mathrm{rt}$")
ax2.set_title(r"$\varepsilon_\mathrm{rt}$")
ax2.legend()
ax3.set_title(r"$2\pi\varepsilon_\mathrm{rt}/\sigma_\mathrm{rt}$")

#fig.legend(lines, line_names, frameon=False, ncol=1, loc='right')
#simplify_axes(ax1)

plt.savefig('white_matter_review.pdf')